In [2]:
#import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys

from pandas import DataFrame, read_csv

%matplotlib tk

In [3]:
#load data
# file path of csv file
Location = r'/Users/33614/ExternalDrive/SUPAERO/PIR_2A/Data/data_spikes/E18KABaseline_Bcut.txt'
#Location = r'/Users/SYL21/D_Drive/SUPAERO/PIR_2A/Data/data_spikes/E18KABaseline_Bcut.txt'
#Location = r'/Users/louiseplacidet/Desktop/PIR/Data/data_spikes/E18KABaseline_Bcut.txt'

# create dataframe
df = pd.read_csv(Location, sep='\t',skiprows=[0,1,3] , index_col='%t           ')

C:\Users\33614\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
plt.figure()
plt.plot(df.index, df.iloc[:,1])
plt.xlabel('Time')
plt.grid()

In [5]:
#threshold

def test_valid_window(window, test_level = 5):
    """
    window : the window in the signal that has to be tested
    
    This funtion test the window to insure that it doesn't contain the signal of interest (spike)
    """
    #non zero ?
    second = np.percentile(window, 2)
    thirtyth = np.percentile(window, 30)
    #print(str(second) + "\t" + str(thirtyth) + "\t" + str(second/thirtyth))
    if abs(second/thirtyth) < test_level : 
        return True
    else : 
        return False
    

def init_noise_levels(signal, fs, 
                      noise_window_size = 0.01,
                      required_valid_windows = 100,
                      old_noise_level_propagation = 0.8, 
                      test_level = 5):
    
    nb_valid_windows = 0
    list_RMS = []
    noise_levels = []
     
    #boucle en indice
#    for window_index in range(0,len(signal)-(len(signal)%int(fs*noise_window_size)),int(fs*noise_window_size)):
    for window_index in range(0,len(signal),int(fs*noise_window_size)):
        test = test_valid_window(signal.iloc[window_index: window_index + int(fs*noise_window_size)], test_level)
        if nb_valid_windows < required_valid_windows:
            if test == True :
                RMS = np.sqrt(np.mean(signal.iloc[window_index: window_index + int(fs*noise_window_size)]**2))
                list_RMS.append(RMS)
                nb_valid_windows += 1
            
            if nb_valid_windows == required_valid_windows:
                noise_level = np.percentile(list_RMS, 25)
                for elm in range(0, window_index, int(fs*noise_window_size)):
                    noise_levels.append(noise_level)
                
        else :
            """if test == True:
                if (window + int(fs*noise_window_size)) > (len(signal)-1) :
                    N25 = np.percentile(abs(signal.iloc[window:]), 25)
                else :
                    N25 = np.percentile(abs(signal.iloc[window: window + int(fs*noise_window_size)]), 25)
                noise_level = old_noise_level_propagation*noise_level + (1-old_noise_level_propagation)*N25
            noise_levels.append(noise_level)"""
            if test == True:
                if (window_index + int(fs*noise_window_size)) > (len(signal)-1) :
                    RMS = np.sqrt(np.mean(signal.iloc[window_index:]**2))
                else :
                    RMS = np.sqrt(np.mean(signal.iloc[window_index: window_index + int(fs*noise_window_size)]**2))
                list_RMS.append(RMS)
                N25 = np.percentile(list_RMS, 25)
                new_noise_level = old_noise_level_propagation*noise_level + (1-old_noise_level_propagation)*N25
                noise_level = new_noise_level
            noise_levels.append(noise_level)
    
    noise_levels.append(noise_level)        
    plt.figure()
    plt.plot(list_RMS)
    plt.xlabel('Time Windows')
    plt.title('RMS values')
    plt.grid(True)
                
    return noise_levels

In [6]:
#find spike

def find_spike(signal, initial_index, noise_levels, fs, spike_centers, 
               window_size = 0.001, 
               noise_window_size = 0.01,
               threshold_factor = 4):
    if initial_index < len(signal):
        i = initial_index
        for value in signal.iloc[initial_index:]:
            if value > threshold_factor*noise_levels[int((i/fs)//noise_window_size)]:
                while(True):
                    if i > initial_index + window_size*fs:
                        b_point = int(i - window_size*fs)
                    else :
                        b_point = initial_index
                    if i < len(signal)-window_size*fs-1:
                        e_point = int(i + window_size*fs)
                    else :
                        e_point = len(signal)-1

                    highest_value = signal.iloc[b_point: e_point].max()
                    #print("highest value:"+str(highest_value))
                    
                    if highest_value == value : # && 50% amplitude
                        #print("Found high spike")
                        #print("index:"+str(i))
                        spike_centers.append(i)
                        return e_point
                    
                    else:
                        #i = signal.index.get_loc(highest)
                        #print("b_point: " + str(b_point))
                        #print("e_point: " + str(e_point))
                        i = int(np.round(signal.iloc[b_point: e_point].idxmax()*fs/1000))#index(highest_value)#+b_pointà
                       #print("i: "+str(i))
                        value = signal.iloc[i]
                        
                break
                    
            if value < -threshold_factor*noise_levels[int((i/fs)//noise_window_size)]:
                while(True):
                    if i > initial_index + window_size*fs:
                        b_point = int(i - window_size*fs)
                    else :
                        b_point = initial_index
                    if i < len(signal)-window_size*fs-1:
                        e_point = int(i + window_size*fs)
                    else :
                        e_point = len(signal)-1

                    lowest_value = min(signal.iloc[b_point: e_point])
                    #print("lowest value:"+str(lowest_value))
                    #print("value: "+ str(value))

                    if lowest_value == value : # && 50% amplitude
                        #print("Found low spike")
                        #print("index:"+str(i))
                        spike_centers.append(i)                        
                        return e_point
                    
                    else:
                        #i = signal.index.get_loc(lowest)
                        #print("b_point: " + str(b_point))
                        #print("e_point: " + str(e_point))
                        i = int(np.round(signal.iloc[b_point: e_point].idxmin()*fs/1000))#Index.get_loc(lowest_value)#+b_point
                        #print("i: "+str(i))
                        value = signal.iloc[i]
                        
                break
                  
            i += 1
    return -2
            
def find_spikes(signal, noise_levels, fs, 
               window_size = 0.001, 
               noise_window_size = 0.01,
               threshold_factor = 4):
    
    initial = 0
    spike_centers = []
    
    while initial != -2:
        initial = find_spike(signal, initial, noise_levels, fs, spike_centers,
                             window_size = window_size, 
                             noise_window_size = noise_window_size,
                             threshold_factor = threshold_factor)
    
    return spike_centers

In [7]:
#record spike

def record_spikes(signal, fs, spike_centers,
                  t_before = 0.001,
                  t_after = 0.002):
    
    data = np.array([[float(x) for x in range(int(np.round(fs*(t_before+t_after)))+1)]])
    
    for center in spike_centers:
        if center < int(np.round(fs*t_before)):
            spike = [0 for i in range(0, int(np.round(fs*t_before-center)))]
            #print('spike before concat:'+ str(len(spike)))
            #spike = np.concatenate(spike, signal.iloc[:center + int(np.round(fs*t_after))].values())
            spike = np.concatenate(spike, signal.values[:center + int(np.round(fs*t_after))])
            #print('spike after concat:'+str(len(spike)))
            data = np.insert(data, len(data), spike, axis=0)
            
        elif center > int(np.round(len(signal)-fs*t_after)):
            #spike = signal.iloc[int(np.round(center-fs*t_before)):].values()
            spike = signal.values[int(np.round(center-fs*t_before)):]
            #print('spike before concat:'+str(len(spike)))
            spike = np.concatenate(spike,[0 for i in range(0, int(np.round(fs*t_after-(len(signal)-center))))])
            #print('spike after concat:'+str(len(spike)))
            data = np.insert(data, len(data), spike, axis=0)
            
        else :
            #spike = signal.iloc[int(np.round(center - fs*t_before)): int(np.round(center + fs*t_after))+1].values()
            spike = signal.values[int(np.round(center - fs*t_before)): int(np.round(center + fs*t_after))+1]
            #print(spike)
            data = np.insert(data, len(data), spike, axis=0)

    print(np.shape(data))
    data = data.transpose()
    #print(data)
    spike_data = pd.DataFrame(data)
    
    return spike_data

In [8]:
signal = df.iloc[:,1]
plt.plot(df.iloc[:,1])
plt.title('Original Signal')
plt.xlabel('Time Windows')
plt.ylabel('Amplitude [µV]')
plt.legend()
plt.grid(True)

In [9]:
# test of the functions

signal = df.iloc[:50000,1]
fs = 25000

noise_levels = init_noise_levels(signal, fs, 
                      noise_window_size = 0.01,
                      required_valid_windows = 100,
                      old_noise_level_propagation = 0.8, 
                      test_level = 5)

C:\Users\33614\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


In [10]:
plt.figure()
plt.plot(noise_levels)
plt.grid(True)
plt.xlabel('Time')
plt.ylabel('Noise Amplitude [µV]')
plt.title('Noise Levels')

Text(0.5,1,'Noise Levels')

In [11]:
spike_centers = find_spikes(signal, noise_levels, fs, 
                           window_size = 0.001, 
                           noise_window_size = 0.01,
                           threshold_factor = 4)

spike_centers

[12616,
 18615,
 18710,
 18740,
 18780,
 18819,
 18854,
 18908,
 18934,
 18959,
 18997,
 19026,
 19054,
 19080,
 19115,
 19169,
 19203,
 19256,
 19292,
 19337,
 19378,
 19452,
 19483,
 19508,
 19568,
 19610,
 19648,
 19690,
 19759,
 19793,
 19823,
 19856,
 19886,
 19928,
 19995,
 20057,
 20098,
 20136,
 20186,
 20218,
 20262,
 20295,
 20344,
 20392,
 20428,
 20517,
 20543,
 20575,
 20603,
 20636,
 20699,
 20742,
 20800,
 20841,
 20873,
 20938,
 21001,
 21039,
 21093,
 21190,
 21232,
 21263,
 21303,
 21331,
 21359,
 21464,
 21533,
 21575,
 21608,
 21638,
 21694,
 21722,
 21821,
 21869,
 21902,
 21939,
 21995,
 22020,
 22053,
 22099,
 22192,
 22249,
 22281,
 22362,
 22387,
 22432,
 22464,
 22491,
 22568,
 22622,
 22717,
 22743,
 22779,
 22838,
 22867,
 22895,
 22987,
 23041,
 23108,
 23175,
 23202,
 23260,
 23303,
 23358,
 23414,
 23455,
 23485,
 23515,
 23575,
 23689,
 23749,
 23872,
 23899,
 23926,
 24040,
 24138,
 24212,
 24247,
 24323,
 24384,
 24409,
 24446,
 24514,
 24549,
 24580,


In [12]:
spike_data = record_spikes(signal, fs, spike_centers, 
                  t_before = 0.001, 
                  t_after = 0.002)

(198, 76)


In [13]:
plt.figure(figsize = (100,20))
plt.plot(spike_data.index, spike_data.iloc[:,1])
plt.xlabel('Time')
plt.grid()

In [28]:
def record_spikes_oneline(signal, fs, spike_centers,
                  t_before = 0.001,
                  t_after = 0.002):
    
    data = np.array(['NaN' for x in range(len(signal))])
    data = data.astype(float)
    times = np.array(['NaN' for x in range(len(signal))])
    times = times.astype(pd.Timestamp)
    
    for center in spike_centers:
        if center < int(np.round(fs*t_before)):
            data[:center + int(np.round(fs*t_after))] = signal.values[:center + int(np.round(fs*t_after))]
            times[:center + int(np.round(fs*t_after))] = signal.index[:center + int(np.round(fs*t_after))]
            
        elif center > int(np.round(len(signal)-fs*t_after)):
            data[int(np.round(center-fs*t_before)):] = signal.values[int(np.round(center-fs*t_before)):]
            times[int(np.round(center-fs*t_before)):] = signal.index[int(np.round(center-fs*t_before)):]
            
        else :
            data[int(np.round(center - fs*t_before)): int(np.round(center + fs*t_after))+1] = signal.values[int(np.round(center - fs*t_before)): int(np.round(center + fs*t_after))+1]
            times[int(np.round(center - fs*t_before)): int(np.round(center + fs*t_after))+1] = signal.index[int(np.round(center - fs*t_before)): int(np.round(center + fs*t_after))+1]

    spike_data_oneline = pd.DataFrame(data, index = times.astype(float))
    
    return spike_data_oneline

In [29]:
spike_data_oneline = record_spikes_oneline(signal, fs, spike_centers,
                  t_before = 0.001,
                  t_after = 0.002)

In [30]:
spike_data_oneline = spike_data_oneline.astype(float)
test = spike_data_oneline.dropna()
test.head()

,0
503.64,0.57
503.68,1.02
503.72,1.48
503.76,1.93
503.80,1.93


In [31]:
signal = df.iloc[:,1]
#signal.plot(color = 'blue')
#spike_data_oneline.plot(color = 'red')
plt.plot(signal.index, signal)
plt.plot(spike_data_oneline.index, spike_data_oneline, color = 'red')
plt.title('Original Signal')
plt.xlabel('Time Windows')
plt.ylabel('Amplitude [µV]')
plt.legend()
plt.grid(True)

In [19]:
signal.index

Float64Index([      0.0,      0.04,      0.08,      0.12,      0.16,       0.2,
                   0.24,      0.28,      0.32,      0.36,
              ...
               301799.6, 301799.64, 301799.68, 301799.72, 301799.76,  301799.8,
              301799.84, 301799.88, 301799.92, 301799.96],
             dtype='float64', name='%t           ', length=7545000)

In [24]:
spike_data_oneline.index.astype(float)

Float64Index([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
              ...
              nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
             dtype='float64', length=50000)